# sm'

In [ ]:
// // test

open testing

()



In [ ]:
open rust_operators

()



In [ ]:
inl types () =
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"str\")>] type Str = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"base64::DecodeError\")>] type base64_DecodeError = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"borsh::io::Error\")>] type borsh_io_Error = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"js_sys::JsString\")>] type js_sys_JsString = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"serde_json::Error\")>] type serde_json_Error = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"serde_json::Value\")>] type serde_json_Value = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"serde_wasm_bindgen::Error\")>] type serde_wasm_bindgen_Error = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::str::Utf8Error\")>] type std_str_Utf8Error = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::string::String\")>] type std_string_String = class end"

()



In [ ]:
// // test

inl types () =
    rust.types ()
    types ()

()



In [ ]:
nominal base64_decode_error = $"base64_DecodeError"
nominal borsh_io_error = $"borsh_io_Error"
nominal utf8_error = $"std_str_Utf8Error"
nominal json_value = $"serde_json_Value"
nominal json_error = $"serde_json_Error"
nominal serde_wasm_bindgen_error = $"serde_wasm_bindgen_Error"
nominal std_string = $"std_string_String"
nominal js_string = $"js_sys_JsString"
nominal str = $"Str"

()



## contains

In [ ]:
inl contains (value : string) (s : string) : bool =
    $"!s.Contains !value"

()



## ends_with

In [ ]:
inl ends_with (value : string) (s : string) : bool =
    $"!s.EndsWith !value"

()



## pad_left

In [ ]:
inl pad_left total_width padding_char (s : string) : string =
    $"!s.PadLeft (!total_width, !padding_char)"

()



## pad_right

In [ ]:
inl pad_right (pad : i32) (s : string) : string =
    $"!s.PadRight !pad"

()



## replace

In [ ]:
inl replace (old_value : string) (new_value : string) (s : string) : string =
    $"!s.Replace (!old_value, !new_value)"

()



## split

In [ ]:
inl split (separator : string) (str : string) : array_base string =
    $"!str.Split !separator"

()



## split_string

In [ ]:
inl split_string (separator : array_base string) (str : string) : array_base string =
    $"!str.Split (!separator, StringSplitOptions.None)"

()



## starts_with

In [ ]:
inl starts_with (value : std_string) (s : std_string) : bool =
    inl s = join s
    !\($'"!s.starts_with(&!value)"')

()



## substring

In [ ]:
inl substring forall t {int}. (start : t) (len : t) (str : string) : string =
    $"!str.Substring (!start, !len)"

()



## to_lower

In [ ]:
inl to_lower (input : string) : string =
    $"!input.ToLower ()"

()



## to_upper

In [ ]:
inl to_upper (input : string) : string =
    $"!input.ToUpper ()"

()



## trim

In [ ]:
inl trim (input : string) : string =
    $"!input.Trim ()"

()



## trim_end

In [ ]:
inl trim_end (trim_chars : array_base char) (input : string) : string =
    $"!input.TrimEnd !trim_chars"

()



## trim_start

In [ ]:
inl trim_start (trim_chars : array_base char) (input : string) : string =
    $"!input.TrimStart !trim_chars"

()



## ellipsis

In [ ]:
inl ellipsis forall t {int; number}. (max : t) (s : string) =
    if sm.length s <= max
    then s
    else s |> substring 0 max |> fun x => $'!x + "..."'

()



## raw_string_literal

In [ ]:
inl raw_string_literal (s : string) : rust.ref' str =
    !\($"\"r#\\\"\" + !s + \"\\\"#\"")

()



## raw_string_literal_static

In [ ]:
inl raw_string_literal_static (s : string) : rust.static_ref' str =
    !\($"\"r#\\\"\" + !s + \"\\\"#\"")

()



## include_str

In [ ]:
inl include_str (path : string) : rust.ref' str =
    !\($'"include_str\!(\\\"" + !path + "\\\")"')

()



## as_str

In [ ]:
inl as_str (s : string) : rust.ref' str =
    inl s = join s
    !\($'$"fable_library_rust::String_::LrcStr::as_str(&!s)"')

()



## from_std_string

In [ ]:
inl from_std_string (str : std_string) : string =
    inl str = join str
    !\($'"fable_library_rust::String_::fromString(!str)"')

()



## ref_to_std_string

In [ ]:
inl ref_to_std_string (str : rust.ref' str) : std_string =
    !\($'"String::from(!str)"')

()



## to_std_string

In [ ]:
inl to_std_string (s : string) : std_string =
    inl s = join s
    s |> as_str |> ref_to_std_string

()



## as_str_std

In [ ]:
inl as_str_std (s : std_string) : rust.ref' str =
    inl s = join s
    !\($'"!s.as_str()"')

()



## into_boxed_str

In [ ]:
inl into_boxed_str (s : std_string) : rust.box str =
    !\($'"!s.into_boxed_str()"')

()



## format'

In [ ]:
inl format' x : std_string =
    !\\(x, $'@@$"format\!(""{{}}"", $0)"')

()



## format_debug

In [ ]:
inl format_debug x : string =
    $'$"%A{!x}"'

()



## format_debug'

In [ ]:
inl format_debug' x : std_string =
    !\\(x, $'@@$"format\!(""{{:?}}"", $0)"')

()



## format_pretty'

In [ ]:
inl format_pretty' x : std_string =
    !\\(x, $'@@$"format\!(""{{:#?}}"", $0)"')

()



## format_exception

In [ ]:
inl format_exception (ex : exn) : string =
    $'$"{!ex.GetType ()}: {!ex.Message}"'

()



In [ ]:
// // test

fun () => failwith "test"
|> _throws
|> optionm.map sm'.format_exception
|> _assert_eq (Some "System.Exception: test")

type [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : exn
and [<Struct>] US1 =
    | US1_0
    | US1_1 of f1_0 : string
let rec closure0 () () : unit =
    failwith<unit> "test"
and closure1 () (v0 : exn) : US0 =
    US0_1(v0)
and method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : (unit -> unit) = closure0()
    let v1 : US0 = US0_0
    let v2 : (exn -> US0) = closure1()
    let v3 : US0 = try v0 (); v1 with ex -> ex |> v2
    let v9 : US1 =
        match v3 with
        | US0_0 -> (* None *)
            US1_0
        | US0_1(v4) -> (* Some *)
            let v5 : string = $"{v4.GetType ()}: {v4.Message}"
            US1_1(v5)
    let v10 : string = $"%A{v9}"
    System.Console.WriteLine v10
    let v15 : bool =
        match v9 with
        | US1_1(v13) -> (* Some *)
            let v14 : bool = v13 = "System.Exception: test"
            v14
        | _ ->
            false
    let v17 : bool =
        if v15 then
            true
        else
            metho

## format_ellipsis'

In [ ]:
inl format_ellipsis' s =
    s
    |> format_debug
    |> ellipsis 400i32

()



## obj_to_string

In [ ]:
inl obj_to_string x : string =
    $"!x.ToString ()"

()



## to_string any

In [ ]:
instance to_string any =
    obj_to_string

()



## to_string result t u

In [ ]:
instance to_string result t u = fun x =>
    real
        open rust
        typecase (t * u) with
        | string * string =>
            match x with
            | Ok x => x
            | Error x => $'"Error: " + !x + ""' : string
        | std_string * std_string =>
            match x with
            | Ok x => from_std_string x
            | Error x => $'"Error: " + string !x + ""' : string
        | _ => obj_to_string `u x

()



## serialize

In [ ]:
inl serialize forall t. (x : t) : resultm.result' std_string json_error =
    !\($'"serde_json::to_string(&!x)"')

()



## deserialize

In [ ]:
inl deserialize forall t. (json : string) : resultm.result' t std_string =
    inl json = json |> as_str
    !\($'"serde_json::from_str(&!json)"')
    |> resultm.map_error' fun (x : json_error) => x |> format'

()



## borsh_deserialize

In [ ]:
inl borsh_deserialize forall t. (data : array_base u8) : resultm.result' t std_string =
    inl data = data |> am'.as_slice
    !\($'"let mut !data = !data"')
    inl result = !\($'"borsh::BorshDeserialize::deserialize(&mut !data)"')
    result
    |> resultm.map_error' fun (x : borsh_io_error) => x |> format'

()



## deserialize_vec

In [ ]:
inl deserialize_vec (value : json_value) : resultm.result' (am'.vec u8) std_string =
    inl value = join value
    !\($'"serde_json::from_value(!value)"')
    |> resultm.map_error' fun (x : json_error) => x |> format'

()



## encode_uri_component

In [ ]:
inl encode_uri_component (s : std_string) : js_string =
    !\($'"js_sys::encode_uri_component(&!s)"')

()



## strip_prefix

In [ ]:
inl strip_prefix (prefix : char) (s : std_string) : optionm'.option' (rust.ref' str) =
    inl s = join s
    !\($'"!s.strip_prefix(!prefix)"')

()



## base64_decode

In [ ]:
inl base64_decode (s : std_string) : result std_string std_string =
    fun () =>
        inl s = join s
        inl bytes : resultm.result' (am'.vec u8) base64_decode_error =
            !\($'"base64::Engine::decode(&base64::engine::general_purpose::STANDARD, !s)"')
        inl bytes =
            bytes
            |> resultm.map_error' format'
            |> resultm.try'
        inl result : resultm.result' std_string utf8_error =
            !\($'"std::str::from_utf8(&!bytes).map(String::from)"')
        result
        |> resultm.map_error' format'
    |> fun x =>
        join x ()
        |> resultm.unbox

()



## concat_array_trailing

In [ ]:
inl concat_array_trailing separator input =
    ("", input)
    ||> am.fold fun acc (x : string) =>
        $'!acc + !x + !separator + ""'

()



In [ ]:
// // test
// // rust=

;[
    "1"
    "2"
    "3"
]
|> fun x =>
    inl code = (a x : _ i32 _) |> concat_array_trailing "\n"
    code
    |> _assert_eq "1\n2\n3\n"

.rs output:
"1
2
3
"



.fsx:
type Mut0 = {mutable l0 : int32; mutable l1 : string}
let rec method1 (v0 : (string [])) : (string []) =
    v0
and method2 (v0 : int32, v1 : Mut0) : bool =
    let v2 : int32 = v1.l0
    let v3 : bool = v2 < v0
    v3
and method3 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "1"
    let v1 : string = "2"
    let v2 : string = "3"
    let v3 : (string []) = [|v0; v1; v2|]
    let v4 : (string []) = method1(v3)
    let v5 : int32 = v4.Length
    let v6 : string = ""
    let v7 : Mut0 = {l0 = 0; l1 = v6} : Mut0
    while method2(v5, v7) do
        let v9 : int32 = v7.l0
        let v10 : string = v7.l1
        let v11 : string = v4.[int v9]
        let v12 : string = "\n"
        let v13 : string = v10 + v11 + v12 + ""
        let v14 : int32 = v9 + 1
        v7.l0 <- v14
        v7.l1 <- v13
        ()
    let v15 : string = v7.l1
    let v16 : string = $"%A{v15}"
    System.Console.WriteLine v16
    let v18 : bool = v15 = "1\n2\n3\n"
    let v20 : boo

## concat_list_trailing

In [ ]:
inl concat_list_trailing separator input =
    ("", input)
    ||> listm.fold fun acc (x : string) =>
        $'!acc + !x + !separator + ""'

()



In [ ]:
// // test
// // rust=

[
    "1"
    "2"
    "3"
]
|> fun x =>
    inl code = (x : _) |> sm'.concat_list_trailing "\n"
    code
    |> _assert_eq "1\n2\n3\n"

.rs output:
"1
2
3
"



.fsx:
let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = ""
    let v1 : string = "1"
    let v2 : string = "\n"
    let v3 : string = v0 + v1 + v2 + ""
    let v4 : string = "2"
    let v5 : string = v3 + v4 + v2 + ""
    let v6 : string = "3"
    let v7 : string = v5 + v6 + v2 + ""
    let v8 : string = $"%A{v7}"
    System.Console.WriteLine v8
    let v10 : bool = v7 = "1\n2\n3\n"
    let v12 : bool =
        if v10 then
            true
        else
            method1(v10)
    let v13 : string = "1\n2\n3\n"
    let v14 : string = $"__expect / actual: %A{v7} / expected: %A{v13}"
    let v15 : bool = v12 = false
    if v15 then
        failwith<unit> v14
method0()


.rs:
#![allow(dead_code,)]
#![allow(non_camel_case_types,)]
#![allow(non_snake_case,)]
#![allow(non_upper_case_globals,)]
#![allow(unreachable_code,)]
#![allow(unused_attributes,)]
#![allow(unused_imports,)]
#![allow(unused_macros,)]
#![allow(unused_parens,)]
#![allow(unused_variab

## concat_list_heap_trailing

In [ ]:
inl concat_list_heap_trailing separator input =
    ("", input)
    ||> listm.fold fun acc (x : string) =>
        inl acc = acc |> to_std_string
        inl x = x |> as_str
        inl separator = separator |> as_str
        !\($'"!acc + !x + !separator"')
        |> from_std_string

()



In [ ]:
// // test
// // rust=

types ()

[
    "1"
    "2"
    "3"
]
|> fun x =>
    inl code = (x : _) |> sm'.concat_list_heap_trailing "\n"
    code
    |> _assert_eq "1\n2\n3\n"

.rs output:
"1
2
3
"



.fsx:
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>] type Func0<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>] type Func0<'T, 'U> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>] type Box<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>] type Dyn<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>] type Fn<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>] type FnUnit = class end
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>] type FnOnce<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $1)")>] type ActionFn2<'T, 'U> = class end
[<Fable.Core.Erase; Fable.Core.Emit("impl $0")>] type Impl<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("mut $0")>] type Mut<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("&$0")>] type Ref<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("&'static $0")>] type StaticRef<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("MutCell<$0>")>] type MutCe

## concat

In [ ]:
inl concat (a : string) (b : a _ _) : string =
    $"!b |> String.concat !a"

()



## join'

In [ ]:
inl join' (concat : string) (s : a _ string) : string =
    $"System.String.Join (!concat, !s)"

()



## to_char_array

In [ ]:
inl to_char_array (str : string) : a i32 char =
    am.init (str |> sm.length) (fun i => sm.index str i)

()



In [ ]:
// // test

"abc"
|> to_char_array
|> _assert_eq (a ;['a'; 'b'; 'c'])

type Mut0 = {mutable l0 : int32}
let rec method1 (v0 : Mut0) : bool =
    let v1 : int32 = v0.l0
    let v2 : bool = v1 < 3
    v2
and method2 (v0 : (char [])) : (char []) =
    v0
and method3 (v0 : (char []), v1 : (char []), v2 : int32) : bool =
    let v3 : int32 = v0.Length
    let v4 : bool = v2 < v3
    if v4 then
        let v5 : char = v0.[int v2]
        let v6 : char = v1.[int v2]
        let v7 : bool = v5 = v6
        if v7 then
            let v8 : int32 = v2 + 1
            method3(v0, v1, v8)
        else
            false
    else
        true
and method4 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : (char []) = Array.zeroCreate<char> (3)
    let v1 : Mut0 = {l0 = 0} : Mut0
    while method1(v1) do
        let v3 : int32 = v1.l0
        let v4 : char = "abc".[int v3]
        v0.[int v3] <- v4
        let v5 : int32 = v3 + 1
        v1.l0 <- v5
        ()
    let v6 : (char []) = [|'a'; 'b'; 'c'|]
    let v7 : (char []) = method2(v6)
    let v8 : string

## to_char_list

In [ ]:
inl to_char_list (str : string) : list char =
    listm.init (str |> sm.length) (fun (i : i64) => sm.index str i)

()



In [ ]:
// // test

"abc"
|> to_char_list
|> _assert_eq ['a'; 'b'; 'c']

type UH0 =
    | UH0_0 of char * UH0
    | UH0_1
let rec method0 () : unit =
    let v0 : UH0 = UH0_1
    let v1 : UH0 = UH0_0('c', v0)
    let v2 : UH0 = UH0_0('b', v1)
    let v3 : UH0 = UH0_0('a', v2)
    let v4 : string = $"%A{v3}"
    System.Console.WriteLine v4
    let v23 : UH0 = UH0_1
    let v24 : UH0 = UH0_0('c', v23)
    let v25 : UH0 = UH0_0('b', v24)
    let v26 : UH0 = UH0_0('a', v25)
    let v27 : UH0 = UH0_1
    let v28 : UH0 = UH0_0('c', v27)
    let v29 : UH0 = UH0_0('b', v28)
    let v30 : UH0 = UH0_0('a', v29)
    let v31 : string = $"__expect / actual: %A{v26} / expected: %A{v30}"
    ()
method0()

UH0_0 ('a', UH0_0 ('b', UH0_0 ('c', UH0_1)))


## to_string std_string

In [ ]:
open rust
instance to_string std_string = from_std_string

()

